In [32]:
!pip install selenium

In [33]:
!pip install pyyaml

In [34]:
import os
import time
import yaml
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options

from selenium.common.exceptions import TimeoutException

In [35]:
# !pip install webdriver-manager

In [55]:
# from webdriver_manager.firefox import GeckoDriverManager
# browser = webdriver.Firefox(executable_path=GeckoDriverManager().install())

# options = Options()
# options.set_capability("moz:firefoxOptions.binary", "/chemin/vers/firefox/binaire")
# driver = webdriver.Firefox(options=options)

options = webdriver.FirefoxOptions()
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')
options.add_argument('start-maximized')
browser = webdriver.Remote(command_executor='http://192.168.65.4:4444', options=options)
print("fjhgsmjdh")

fjhgsmjdh


In [56]:
# Options
departure = 'Paris'
arrival = 'Tunis'
max_price = 950

# <div class="vvTc-item-value">New York, United States (NYC)</div>

In [57]:
browser.implicitly_wait(10)
browser_wait = WebDriverWait(browser, 30)

url = "https://www.kayak.com/flights/PAR-BKK/2023-03-22/2023-03-29?sort=bestflight_a"
browser.get(url)
time.sleep(3)

In [58]:

print(browser.page_source)


<html class="animation filter Flights-Search-FlightSearchPage-Html" lang="en-us"><head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#">
<script async="" type="text/javascript" src="//www.googletagservices.com/tag/js/gpt.js"></script><script src="/res/js/perf.js?v=b0cedeca48f25274e91e1a7a547dbdcdd5533e28-frpreducers&amp;cluster=5"></script><script type="text/javascript">
(function() {
var style = document.createElement('style');
style.textContent = '@import "https://content.r9cdn.net/res/combined.css?v=bbce429329bb1b171389f80cc5d8bb1dffc44334-frpreducers&cluster=5"; @import "https://content.r9cdn.net/res/combined.css?v=5a0374a53decdd5a83c3d9b5e7097bf704543140-frpreducers&cluster=5"; ';
var fi = setInterval(function() {
try {
style.sheet.cssRules;
document.body.style.display = "block";
clearInterval(fi);
} catch (e){}
}, 10);
document.head.appendChild(style);
})();
</script><style>@import "https://content.r9cdn.net/res/combined.css?v=bbce429329bb1b171389f80cc5d8bb1dffc44334-frpre

In [59]:
# BCCW = "//div[contains(@class,'BCcW')]"
# destination = browser.find_elements(By.XPATH,BCCW)
# destination

In [60]:
# destination[0]

In [61]:
# texte_prix = "//div[@id='V_Fq-mb-aE-1836ce41d9d']//span[@class='price-text']"
# prix = browser.find_elements(By.XPATH,texte_prix) 
# texte = prix.text
# print(texte)

In [62]:
mon_element = browser.find_elements(By.XPATH, "//div[@id='V_Fq-mb-aE-1836ce41d9d']//a[@class='booking-link']//span[@class='price-text']")
print(mon_element)

InvalidSessionIdException: Message: Unable to execute request for an existing session: Unable to find session with ID: 9b0eecc3-908f-41a3-a654-f17eeb9c0893
Build info: version: '4.8.0', revision: '267030adea'
System info: os.name: 'Linux', os.arch: 'amd64', os.version: '5.10.16.3-microsoft-standard-WSL2', java.version: '11.0.17'
Driver info: driver.version: unknown


In [ ]:
def load_more():
    try:
        more_results = '//a[@class = "moreButton"]'
        browser.find_element_by_xpath(more_results).click()
        print('sleeping.....')
        sleep(randint(25,35))
    except:
        pass

In [ ]:
def start_kayak(city_from, city_to, date_start, date_end):
    """City codes - it's the IATA codes!
    Date format -  YYYY-MM-DD"""
    
    kayak = ('https://www.kayak.com/flights/' + city_from + '-' + city_to +
             '/' + date_start + '-flexible/' + date_end + '-flexible?sort=bestflight_a')
    browser.get(kayak)
    sleep(randint(8,10))
    
    # sometimes a popup shows up, so we can use a try statement to check it and close
    try:
        xp_popup_close = '//button[contains(@id,"dialog-close") and contains(@class,"Button-No-Standard-Style close ")]'
        browser.find_elements_by_xpath(xp_popup_close)[5].click()
    except Exception as e:
        pass
    sleep(randint(60,95))
    print('loading more.....')
    
#     load_more()
    
    print('starting first scrape.....')
    df_flights_best = page_scrape()
    df_flights_best['sort'] = 'best'
    sleep(randint(60,80))
    
    # Let's also get the lowest prices from the matrix on top
    matrix = browser.find_elements_by_xpath('//*[contains(@id,"FlexMatrixCell")]')
    matrix_prices = [price.text.replace('$','') for price in matrix]
    matrix_prices = list(map(int, matrix_prices))
    matrix_min = min(matrix_prices)
    matrix_avg = sum(matrix_prices)/len(matrix_prices)
    
    print('switching to cheapest results.....')
    cheap_results = '//a[@data-code = "price"]'
    browser.find_element_by_xpath(cheap_results).click()
    sleep(randint(60,90))
    print('loading more.....')
    
#     load_more()

    print('starting second scrape.....')
    df_flights_cheap = page_scrape()
    df_flights_cheap['sort'] = 'cheap'
    sleep(randint(60,80))
    
    print('switching to quickest results.....')
    quick_results = '//a[@data-code = "duration"]'
    browser.find_element_by_xpath(quick_results).click()  
    sleep(randint(60,90))
    print('loading more.....')
    
#     load_more()
    
    print('starting third scrape.....')
    df_flights_fast = page_scrape()
    df_flights_fast['sort'] = 'fast'
    sleep(randint(60,80))
    
    # saving a new dataframe as an excel file. the name is custom made to your cities and dates
    final_df = df_flights_cheap.append(df_flights_best).append(df_flights_fast)
    final_df.to_excel('search_backups//{}_flights_{}-{}_from_{}_to_{}.xlsx'.format(strftime("%Y%m%d-%H%M"),
                                                                                   city_from, city_to, 
                                                                                   date_start, date_end), index=False)
    print('saved df.....')
    
    # We can keep track of what they predict and how it actually turns out!
    xp_loading = '//div[contains(@id,"advice")]'
    loading = browser.find_element_by_xpath(xp_loading).text
    xp_prediction = '//span[@class="info-text"]'
    prediction = browser.find_element_by_xpath(xp_prediction).text
    print(loading+'\n'+prediction)
    
    # sometimes we get this string in the loading variable, which will conflict with the email we send later
    # just change it to "Not Sure" if it happens
    weird = '¯\\_(ツ)_/¯'
    if loading == weird:
        loading = 'Not sure'
    
    username = 'YOUREMAIL@hotmail.com'
    password = 'YOUR PASSWORD'
    
    server = smtplib.SMTP('smtp.outlook.com', 587)
    server.ehlo()
    server.starttls()
    server.login(username, password)
    msg = ('Subject: Flight Scraper\n\n\
Cheapest Flight: {}\nAverage Price: {}\n\nRecommendation: {}\n\nEnd of message'.format(matrix_min, matrix_avg, (loading+'\n'+prediction)))
    message = MIMEMultipart()
    message['From'] = 'YOUREMAIL@hotmail.com'
    message['to'] = 'YOUROTHEREMAIL@domain.com'
    server.sendmail('YOUREMAIL@hotmail.com', 'YOUROTHEREMAIL@domain.com', msg)
    print('sent email.....')

In [ ]:
def page_scrape():
    """This function takes care of the scraping part"""
    
    xp_sections = '//*[@class="section duration"]'
    sections = browser.find_elements_by_xpath(xp_sections)
    sections_list = [value.text for value in sections]
    section_a_list = sections_list[::2] # This is to separate the two flights
    section_b_list = sections_list[1::2] # This is to separate the two flights
    
    # if you run into a reCaptcha, you might want to do something about it
    # you will know there's a problem if the lists above are empty
    # this if statement lets you exit the bot or do something else
    # you can add a sleep here, to let you solve the captcha and continue scraping
    # i'm using a SystemExit because i want to test everything from the start
    if section_a_list == []:
        raise SystemExit
    
    # I'll use the letter A for the outbound flight and B for the inbound
    a_duration = []
    a_section_names = []
    for n in section_a_list:
        # Separate the time from the cities
        a_section_names.append(''.join(n.split()[2:5]))
        a_duration.append(''.join(n.split()[0:2]))
    b_duration = []
    b_section_names = []
    for n in section_b_list:
        # Separate the time from the cities
        b_section_names.append(''.join(n.split()[2:5]))
        b_duration.append(''.join(n.split()[0:2]))

    xp_dates = '//div[@class="section date"]'
    dates = browser.find_elements_by_xpath(xp_dates)
    dates_list = [value.text for value in dates]
    a_date_list = dates_list[::2]
    b_date_list = dates_list[1::2]
    # Separating the weekday from the day
    a_day = [value.split()[0] for value in a_date_list]
    a_weekday = [value.split()[1] for value in a_date_list]
    b_day = [value.split()[0] for value in b_date_list]
    b_weekday = [value.split()[1] for value in b_date_list]
    
    # getting the prices
    xp_prices = '//a[@class="booking-link"]/span[@class="price option-text"]'
    prices = browser.find_elements_by_xpath(xp_prices)
    prices_list = [price.text.replace('$','') for price in prices if price.text != '']
    prices_list = list(map(int, prices_list))

    # the stops are a big list with one leg on the even index and second leg on odd index
    xp_stops = '//div[@class="section stops"]/div[1]'
    stops = browser.find_elements_by_xpath(xp_stops)
    stops_list = [stop.text[0].replace('n','0') for stop in stops]
    a_stop_list = stops_list[::2]
    b_stop_list = stops_list[1::2]

    xp_stops_cities = '//div[@class="section stops"]/div[2]'
    stops_cities = browser.find_elements_by_xpath(xp_stops_cities)
    stops_cities_list = [stop.text for stop in stops_cities]
    a_stop_name_list = stops_cities_list[::2]
    b_stop_name_list = stops_cities_list[1::2]
    
    # this part gets me the airline company and the departure and arrival times, for both legs
    xp_schedule = '//div[@class="section times"]'
    schedules = browser.find_elements_by_xpath(xp_schedule)
    hours_list = []
    carrier_list = []
    for schedule in schedules:
        hours_list.append(schedule.text.split('\n')[0])
        carrier_list.append(schedule.text.split('\n')[1])
    # split the hours and carriers, between a and b legs
    a_hours = hours_list[::2]
    a_carrier = carrier_list[::2]
    b_hours = hours_list[1::2]
    b_carrier = carrier_list[1::2]

    
    cols = (['Out Day', 'Out Time', 'Out Weekday', 'Out Airline', 'Out Cities', 'Out Duration', 'Out Stops', 'Out Stop Cities',
            'Return Day', 'Return Time', 'Return Weekday', 'Return Airline', 'Return Cities', 'Return Duration', 'Return Stops', 'Return Stop Cities',
            'Price'])

    drivflights_df = pd.DataFrame({'Out Day': a_day,
                               'Out Weekday': a_weekday,
                               'Out Duration': a_duration,
                               'Out Cities': a_section_names,
                               'Return Day': b_day,
                               'Return Weekday': b_weekday,
                               'Return Duration': b_duration,
                               'Return Cities': b_section_names,
                               'Out Stops': a_stop_list,
                               'Out Stop Cities': a_stop_name_list,
                               'Return Stops': b_stop_list,
                               'Return Stop Cities': b_stop_name_list,
                               'Out Time': a_hours,
                               'Out Airline': a_carrier,
                               'Return Time': b_hours,
                               'Return Airline': b_carrier,                           
                               'Price': prices_list})[cols]
    
    flights_df['timestamp'] = strftime("%Y%m%d-%H%M") # so we can know when it was scraped
    return flights_df

In [ ]:
# city_from = input('From which city? ')
# city_to = input('Where to? ')
# date_start = input('Search around which departure date? Please use YYYY-MM-DD format only ')
# date_end = input('Return when? Please use YYYY-MM-DD format only ')

city_from = 'LIS'
city_to = 'SIN'
date_start = '2019-08-21'
date_end = '2019-09-07'

for n in range(0,5):
    start_kayak(city_from, city_to, date_start, date_end)
    print('iteration {} was complete @ {}'.format(n, strftime("%Y%m%d-%H%M")))
    
    # Wait 4 hours
    sleep(60*60*4)
    print('sleep finished.....')

In [ ]:
# destination = browser.find_elements(By.XPATH,"//div[starts-with(@class, 'BCcW')]")
destination = browser.find_elements("BCcW")

# destination.send_keys("Tunis")

In [ ]:
# Go to Flights
with open('search_page.yaml', 'r') as f:
    search_page_selectors = yaml.load(f, Loader=yaml.SafeLoader)

time.sleep(1)
try:
    cookies_xpath = search_page_selectors['accept_cookies_xpath']
    WebDriverWait(browser, 5).until(EC.element_to_be_clickable((By.XPATH, cookies_xpath))).click()
except TimeoutException:
    pass

time.sleep(1)

In [ ]:
# From
browser.find_element_by_xpath(search_page_selectors['from_click_xpath']).click()
browser.find_element_by_xpath(search_page_selectors['from_text_xpath']).send_keys(Keys.BACKSPACE + Keys.BACKSPACE + departure)
time.sleep(0.5)
browser.find_element_by_xpath(search_page_selectors['from_text_xpath']).send_keys(Keys.RETURN)
time.sleep(1)

In [ ]:
# To
browser.find_element_by_xpath(search_page_selectors['to_click_xpath']).click()
browser.find_element_by_xpath(search_page_selectors['to_text_xpath']).send_keys(arrival)
time.sleep(0.5)
browser.find_element_by_xpath(search_page_selectors['to_text_xpath']).send_keys(Keys.RETURN)
time.sleep(1)

In [ ]:
# From Date
browser.find_element_by_xpath(search_page_selectors['date_picker_from_xpath']).click()
time.sleep(1)
Select(browser.find_element_by_xpath(search_page_selectors['from_flexible_xpath'])).select_by_value('plusminusthree')
Select(browser.find_element_by_xpath(search_page_selectors['to_flexible_xpath'])).select_by_value('plusminusthree')

In [ ]:
browser.find_element_by_xpath(search_page_selectors['from_date_click_xpath']).click()
browser.find_element_by_xpath(search_page_selectors['from_date_text_xpath']).clear()
browser.find_element_by_xpath(search_page_selectors['from_date_text_xpath']).send_keys('05/08/2021')
time.sleep(1)
browser.find_element_by_xpath(search_page_selectors['to_date_text_xpath']).clear()
time.sleep(1)
browser.find_element_by_xpath(search_page_selectors['to_date_text_xpath']).send_keys('13/08/2021')
time.sleep(1)
browser.find_element_by_xpath(search_page_selectors['submit_button_xpath']).click()

In [ ]:
from selenium.webdriver.common.keys import Keys
# From
from_click_xpath = '//*[contains(@id, "origin-airport-display")]'
from_text_xpath = '//input[contains(@id, "origin-airport")]'
departure = 'Paris'
browser.find_element_by_xpath(from_click_xpath).click()
browser.find_element_by_xpath(from_text_xpath).send_keys(Keys.BACKSPACE + Keys.BACKSPACE + departure)
time.sleep(0.5)
browser.find_element_by_xpath(from_text_xpath).send_keys(Keys.RETURN)
time.sleep(1)
# To
to_click_xpath = '//*[contains(@id, "destination-airport-display")]'
to_text_xpath = r'//input[contains(@id, "destination-airport")]'
arrival = 'Tunis'
browser.find_element_by_xpath(to_click_xpath).click()
browser.find_element_by_xpath(to_text_xpath).send_keys(arrival)
time.sleep(0.5)
browser.find_element_by_xpath(to_text_xpath).send_keys(Keys.RETURN)
time.sleep(1)